<a href="https://colab.research.google.com/gist/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e/untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music genre classification notebook

## Importing Libraries

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [4]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [5]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [7]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

The data has been extracted into a [data.csv](https://github.com/parulnith/Music-Genre-Classification-with-Python/blob/master/data.csv) file.

# Analysing the Data in Pandas

In [8]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.130225,1784.420446,2002.650192,3806.485316,0.083066,-113.596748,121.557297,-19.158825,...,8.810669,-3.667369,5.751690,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
1,blues.00001.wav,0.340983,0.095918,1529.835316,2038.617579,3548.820207,0.056044,-207.556793,124.006721,8.930560,...,5.376803,-2.239120,4.216963,-6.012273,0.936110,-0.716537,0.293876,-0.287431,0.531574,blues
2,blues.00002.wav,0.363603,0.175573,1552.481958,1747.165985,3040.514948,0.076301,-90.754387,140.459915,-29.109968,...,5.789265,-8.905224,-1.083720,-9.218360,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,0.141191,1070.119953,1596.333948,2185.028454,0.033309,-199.431152,150.099213,5.647593,...,6.087676,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637981,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,0.091563,1835.494603,1748.362448,3580.945013,0.101500,-160.266037,126.198807,-35.605450,...,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703780,blues


In [9]:
data.shape

(1000, 28)

In [10]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [11]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [12]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
len(y_train)

800

In [15]:
len(y_test)

200

In [16]:
X_train[10]

array([-1.25331176, -0.45230755, -1.24707319, -0.68362997, -1.14842589,
       -1.53644158, -1.44055293,  0.20268787,  1.27804184, -0.75663826,
        0.56086202,  0.92228   , -0.83907232,  0.22037373, -0.59758985,
       -0.77574318, -1.41075324, -1.65876215, -1.1222043 , -0.94545654,
       -0.72049141, -3.67975559, -1.53705853, -2.94394806, -4.30919491,
       -4.69696269])

# Classification with Keras

## Building our Network

In [18]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [19]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                   

Train on 800 samples
Epoch 1/20
800/800 [==============================] - 0s 501us/sample - loss: 2.1805 - accuracy: 0.2412
Epoch 2/20
800/800 [==============================] - 0s 11us/sample - loss: 1.8851 - accuracy: 0.3475
Epoch 3/20
800/800 [==============================] - 0s 12us/sample - loss: 1.6752 - accuracy: 0.3963
Epoch 4/20
800/800 [==============================] - 0s 12us/sample - loss: 1.5020 - accuracy: 0.4812
Epoch 5/20
800/800 [==============================] - 0s 12us/sample - loss: 1.3658 - accuracy: 0.5175
Epoch 6/20
800/800 [==============================] - 0s 11us/sample - loss: 1.2466 - accuracy: 0.5675
Epoch 7/20
800/800 [==============================] - 0s 12us/sample - loss: 1.1547 - accuracy: 0.6137
Epoch 8/20
800/800 [==============================] - 0s 14us/sample - loss: 1.0646 - accuracy: 0.6413
Epoch 9/20
800/800 [==============================] - 0s 11us/sample - loss: 0.9996 - accuracy: 0.6575
Epoch 10/20
800/800 [==============================

In [21]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [22]:
print('test_acc: ',test_acc)

test_acc:  0.685


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [23]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

Now let's train our network for 20 epochs:

In [24]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 0s 774us/sample - loss: 2.2974 - accuracy: 0.1117 - val_loss: 2.1378 - val_accuracy: 0.3300
Epoch 2/30
600/600 [==============================] - 0s 46us/sample - loss: 2.0950 - accuracy: 0.3233 - val_loss: 2.0375 - val_accuracy: 0.3850
Epoch 3/30
600/600 [==============================] - 0s 23us/sample - loss: 1.9765 - accuracy: 0.3717 - val_loss: 1.9347 - val_accuracy: 0.3800
Epoch 4/30
600/600 [==============================] - 0s 23us/sample - loss: 1.8670 - accuracy: 0.4050 - val_loss: 1.8012 - val_accuracy: 0.4050
Epoch 5/30
600/600 [==============================] - 0s 23us/sample - loss: 1.7216 - accuracy: 0.4283 - val_loss: 1.7022 - val_accuracy: 0.4150
Epoch 6/30
600/600 [==============================] - 0s 23us/sample - loss: 1.6048 - accuracy: 0.4917 - val_loss: 1.6216 - val_accuracy: 0.4600
Epoch 7/30
600/600 [==============================] - 0s 23us/sample - loss: 1.4930

In [25]:
results

[0.9872125506401062, 0.665]

## Predictions on Test Data

In [26]:
predictions = model.predict(X_test)

In [27]:
predictions[0].shape

(10,)

In [28]:
np.sum(predictions[0])

0.99999994

In [29]:
np.argmax(predictions[0])

7

In [31]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=10,init='k-means++', random_state=0).fit(X)

In [32]:
test=pd.DataFrame()
test['y']=y
test['ny']=kmeans.labels_
test.head()
groupedtest=pd.DataFrame(test.groupby(by=['y','ny']).size().reset_index(name='count'))

In [33]:
from sklearn.metrics.cluster import adjusted_rand_score
print(adjusted_rand_score(test.y,test.ny))

0.15677380790973333
